In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
import folium
from fuzzywuzzy import fuzz
from prettytable import PrettyTable
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.set_option("display.float_format", "{:.2f}".format)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/countries-of-the-world-2023/world-data-2023.csv


# **Loading Data**

In [2]:
df=pd.read_csv('/kaggle/input/countries-of-the-world-2023/world-data-2023.csv')

# **Understand the Data**

In [3]:
df.head(5)

,Country,Density\n(P/Km2),Abbreviation,Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,...,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude
0,Afghanistan,60,AF,58.10%,"652,230","323,000",32.49,93.00,Kabul,"8,672",...,78.40%,0.28,"38,041,754",48.90%,9.30%,71.40%,11.12%,"9,797,273",33.94,67.71
1,Albania,105,AL,43.10%,"28,748","9,000",11.78,355.00,Tirana,"4,536",...,56.90%,1.20,"2,854,191",55.70%,18.60%,36.60%,12.33%,"1,747,593",41.15,20.17
2,Algeria,18,DZ,17.40%,"2,381,741","317,000",24.28,213.00,Algiers,"150,006",...,28.10%,1.72,"43,053,054",41.20%,37.20%,66.10%,11.70%,"31,510,100",28.03,1.66
3,Andorra,164,AD,40.00%,468,NaN,7.20,376.00,Andorra la Vella,469,...,36.40%,3.33,"77,142",NaN,NaN,NaN,NaN,"67,873",42.51,1.52
4,Angola,26,AO,47.50%,"1,246,700","117,000",40.73,244.00,Luanda,"34,693",...,33.40%,0.21,"31,825,295",77.50%,9.20%,49.10%,6.89%,"21,061,025",-11.20,17.87


In [4]:
df.shape

(195, 35)

In [5]:
df.columns

Index(['Country', 'Density\n(P/Km2)', 'Abbreviation', 'Agricultural Land( %)',
       'Land Area(Km2)', 'Armed Forces size', 'Birth Rate', 'Calling Code',
       'Capital/Major City', 'Co2-Emissions', 'CPI', 'CPI Change (%)',
       'Currency-Code', 'Fertility Rate', 'Forested Area (%)',
       'Gasoline Price', 'GDP', 'Gross primary education enrollment (%)',
       'Gross tertiary education enrollment (%)', 'Infant mortality',
       'Largest city', 'Life expectancy', 'Maternal mortality ratio',
       'Minimum wage', 'Official language', 'Out of pocket health expenditure',
       'Physicians per thousand', 'Population',
       'Population: Labor force participation (%)', 'Tax revenue (%)',
       'Total tax rate', 'Unemployment rate', 'Urban_population', 'Latitude',
       'Longitude'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 35 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Country                                    195 non-null    object 
 1   Density
(P/Km2)                            195 non-null    object 
 2   Abbreviation                               188 non-null    object 
 3   Agricultural Land( %)                      188 non-null    object 
 4   Land Area(Km2)                             194 non-null    object 
 5   Armed Forces size                          171 non-null    object 
 6   Birth Rate                                 189 non-null    float64
 7   Calling Code                               194 non-null    float64
 8   Capital/Major City                         192 non-null    object 
 9   Co2-Emissions                              188 non-null    object 
 10  CPI                       

In [7]:
df.isnull().sum()

Country                                       0
Density\n(P/Km2)                              0
Abbreviation                                  7
Agricultural Land( %)                         7
Land Area(Km2)                                1
Armed Forces size                            24
Birth Rate                                    6
Calling Code                                  1
Capital/Major City                            3
Co2-Emissions                                 7
CPI                                          17
CPI Change (%)                               16
Currency-Code                                15
Fertility Rate                                7
Forested Area (%)                             7
Gasoline Price                               20
GDP                                           2
Gross primary education enrollment (%)        7
Gross tertiary education enrollment (%)      12
Infant mortality                              6
Largest city                            

# **Data Cleaning**

In [8]:
df.Country.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'The Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Ivory Coast',
       'Cape Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Republic of the Congo', 'Costa Rica', 'Croatia',
       'Cuba', 'Cyprus', 'Czech Republic',
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'The Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada',
       'Guatemal

In [9]:
#Replacing Country Name 
df.Country=df.Country.str.replace("S�����������","Sao Tome and Principe")

In [10]:
df.Country.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'The Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Ivory Coast',
       'Cape Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Republic of the Congo', 'Costa Rica', 'Croatia',
       'Cuba', 'Cyprus', 'Czech Republic',
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'The Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada',
       'Guatemal

In [11]:
df.Country.count()

195

In [12]:
# Cleaning Data and Changing data type from str to float
column_to_float=['Density\n(P/Km2)', 'Agricultural Land( %)','Land Area(Km2)',
                 'Birth Rate', 'Co2-Emissions', 'Forested Area (%)',
                 'CPI', 'CPI Change (%)', 'Fertility Rate', 'Gasoline Price','GDP',
                 'Gross primary education enrollment (%)', "Armed Forces size",
                 'Gross tertiary education enrollment (%)', 'Infant mortality',
                 'Life expectancy', 'Maternal mortality ratio','Minimum wage', 
                 'Out of pocket health expenditure','Physicians per thousand', 
                 'Population','Population: Labor force participation (%)', 
                 'Tax revenue (%)','Total tax rate', 'Unemployment rate', 'Urban_population']

for column in column_to_float:
    df[column]=df[column].astype(str)
    df[column]=df[column].str.replace(",","")
    df[column]=df[column].str.replace("$","", regex=False)
    df[column]=df[column].str.replace("%","").astype(float)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 35 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Country                                    195 non-null    object 
 1   Density
(P/Km2)                            195 non-null    float64
 2   Abbreviation                               188 non-null    object 
 3   Agricultural Land( %)                      188 non-null    float64
 4   Land Area(Km2)                             194 non-null    float64
 5   Armed Forces size                          171 non-null    float64
 6   Birth Rate                                 189 non-null    float64
 7   Calling Code                               194 non-null    float64
 8   Capital/Major City                         192 non-null    object 
 9   Co2-Emissions                              188 non-null    float64
 10  CPI                       

In [14]:
#Palestinian National Authority, we have almost anything data about it, so I'll remove it from our sample

dfPalestine=df[df['Country']=='Palestinian National Authority']
dfPalestine

,Country,Density\n(P/Km2),Abbreviation,Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,...,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude
133,Palestinian National Authority,847.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.95,35.23


In [15]:
dfPalestine.isnull()

,Country,Density\n(P/Km2),Abbreviation,Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,...,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude
133,False,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,False


In [16]:
# Removing Palestinian National Authority from DF

df=df[df['Country']!='Palestinian National Authority']

In [17]:
df.Country.count()

194

In [18]:
# Even we have some NaN or None values, we'll use Pandas capacity to handle this scenario

## We could complete NaN using mean & mode, but this time we won't do it
## Replace NULL VALUES with mode for object & median for int/float
## def replace_null_values(dataframe):
##    for col in dataframe.columns:
##        if dataframe[col].dtype == 'object':
##           mode_value = mode(dataframe[col], axis =0, keepdims= True).mode[0]
##            mode_value = dataframe[col].mode().iloc[0]
##            dataframe[col].fillna(mode_value, inplace=True)
##        else:
##            median_value = np.nanmedian(dataframe[col])
##            dataframe[col].fillna(median_value, inplace=True)
##    return dataframe

# **Feature Engineering**

In [19]:
#Taking from external Website info about Continent 
continent = pd.read_html(
    "https://statisticstimes.com/geography/countries-by-continents.php",
    match="Continent",
)[0]
continent=continent[['Country or Area','Continent']]

In [20]:
#Assign Continent to Data Frame Countries
matching_threshold = 90  # Adjust this threshold as needed

for index, row in df.iterrows():
    country_name = row['Country']
    best_match = None
    best_score = 0

    for _, continent_row in continent.iterrows():
        score = fuzz.ratio(country_name, continent_row['Country or Area'])
        if score > best_score and score >= matching_threshold:
            best_match = continent_row
            best_score = score

    if best_match is not None:
        # Merge data from best_match into df
        df.loc[index, 'Continent'] = best_match['Continent']


In [21]:
# Data Frame Continent still has Null values
df.loc[df['Continent'].isna(), "Country"]

11                        The Bahamas
20                            Bolivia
24                             Brunei
28                        Ivory Coast
29                         Cape Verde
39              Republic of the Congo
44                     Czech Republic
62                         The Gambia
73                       Vatican City
79                               Iran
81                Republic of Ireland
92                               Laos
111    Federated States of Micronesia
112                           Moldova
127                       North Korea
143                            Russia
162                       South Korea
170                             Syria
172                          Tanzania
174                        East Timor
185                    United Kingdom
186                     United States
190                         Venezuela
191                           Vietnam
Name: Country, dtype: object

In [22]:
# Let's fill it out manually

continent_fill = pd.DataFrame({
  "Country": ["The Bahamas","Bolivia","Brunei","Ivory Coast","Cape Verde",
        "Republic of the Congo","Czech Republic","The Gambia",
        "Vatican City","Iran","Republic of Ireland","Laos",
        "Federated States of Micronesia","Moldova","North Korea",
        "Russia","South Korea","Syria","Tanzania","East Timor",
        "United Kingdom","United States","Venezuela","Vietnam"],
  "Continent fill": ["North America","South America","Asia","Africa","Oceania",
        "Africa","Europe","Africa",
        "Europe","Asia","Europe","Asia",
        "Oceania","Europe","Asia",
        "Europe","Asia","Asia","Africa","Oceania",
        "Europe","North America","South America","Asia"]
})

continent_fill

,Country,Continent fill
0,The Bahamas,North America
1,Bolivia,South America
2,Brunei,Asia
3,Ivory Coast,Africa
4,Cape Verde,Oceania
5,Republic of the Congo,Africa
6,Czech Republic,Europe
7,The Gambia,Africa
8,Vatican City,Europe
9,Iran,Asia


In [23]:
# Merge Original Data Frame with Continent DF
df = pd.merge(df, continent_fill, on="Country", how='left')
df["Continent"] = df["Continent"].fillna(df["Continent fill"])
df = df.drop(columns=["Continent fill"])

In [24]:
#Every Country has Continent assigned
df.loc[df['Continent'].isna(), "Country"]

Series([], Name: Country, dtype: object)

In [25]:
Table=PrettyTable()
Table.field_names=["Continent","Country"]

for index, row in df.iterrows():
    Table.add_row([(row['Continent']),(row['Country'])])

Table.sortby="Continent"
print(Table)

+---------------+----------------------------------+
|   Continent   |             Country              |
+---------------+----------------------------------+
|     Africa    |             Algeria              |
|     Africa    |              Angola              |
|     Africa    |              Benin               |
|     Africa    |             Botswana             |
|     Africa    |           Burkina Faso           |
|     Africa    |             Burundi              |
|     Africa    |             Cameroon             |
|     Africa    |     Central African Republic     |
|     Africa    |               Chad               |
|     Africa    |             Comoros              |
|     Africa    | Democratic Republic of the Congo |
|     Africa    |             Djibouti             |
|     Africa    |              Egypt               |
|     Africa    |        Equatorial Guinea         |
|     Africa    |             Eritrea              |
|     Africa    |             Eswatini        

In [26]:
#Let's create some important Data  
df['GDP Per Capita']=df['GDP']/df['Population']
df['Armed Forces Per Capita']=df['Armed Forces size']/df['Population']
df['Agricultural Land(Km2)']=df['Agricultural Land( %)']*df['Land Area(Km2)']/100
df['Forested Area(Km2)']=df['Forested Area (%)']*df['Land Area(Km2)']/100
df['Urban Population Ratio']=df['Urban_population']/df['Population']
df['Co2-Emissions Per Capita']=df['Co2-Emissions']/df['Population']
df["GDP rank"] = df["GDP"].rank(ascending=False)

df["Continent GDP"] = df.groupby("Continent")["GDP"].transform("sum")
df["GDP Per Country Per Cont."] = (df["GDP"] / df["Continent GDP"])
Total_Population = df["Population"].sum()
df["Population_Percentage"] = (df["Population"] / Total_Population) * 100

df["Cont_Population"] = df.groupby("Continent")["Population"].transform("sum")
df["Cont_Density"] = df.groupby("Continent")["Density\n(P/Km2)"].transform("mean")
df["Cont_Agricultural_Land"] = df.groupby("Continent")["Agricultural Land( %)"].transform("mean")
df["Cont_LandArea(Km2)"] = df.groupby("Continent")["Land Area(Km2)"].transform("sum")
df["Cont_ArmedForcesSize"] = df.groupby("Continent")["Armed Forces size"].transform("sum")
df["Cont_Co2-Emissions"] = df.groupby("Continent")["Co2-Emissions"].transform("mean")

In [27]:
#Let's use Thematic Map 
def choroplethMap(column, color = "RdBu", use_log = False):
    if use_log:
        values = np.log(df[column])
        label = f"Log {column}"
    else:
        label = column
        values = df[column].values
    fig = px.choropleth(
        df,
        locations="Country",
        locationmode="country names",
        color=values,
        #hover_name="Country",
        labels={"color": label},
        color_continuous_scale=color,
        width=800
    )
    fig.update_layout(
        title=column.replace("_", " "),
        title_font={"size": 20},
        margin=dict(t=50, b=50, l=0, r=0),
    )
    fig.show()
    return None

In [28]:
#FUNCTION RETURNS TOP VALUES FROM 2 COLUMNS
def Top_Values(column1,column2,top):
    cols=df[[column1,column2]].dropna().sort_values(by=column2,
      ascending=False).head(top)

#cols = ["Country","Life expectancy"]

# Create an instance of PrettyTable and set the field names
    table = PrettyTable()
    table.title=f"Top values {column1} vs {column2}"
    table.field_names = cols

# Find the row with the maximum GDP value for each continent
#max_gdp_each_cont = df.groupby("Country")["Life expectancy"].idxmax()

# Extract and add the rows with the maximum GDP values to the table
    for index, row in cols.iterrows():
        table.add_row([row[column1], row[column2]])

    # Set the float format for the specified column
    table.float_format[column2] = ".2"

    
    return table

In [29]:
#FUNCTION RETURNS BOTTOM VALUES FROM 2 COLUMNS
def Bottom_Values(column1,column2,bottom):
    cols=df[[column1,column2]].dropna().sort_values(by=column2,
      ascending=True).head(bottom)

#cols = ["Country","Life expectancy"]

# Create an instance of PrettyTable and set the field names
    table = PrettyTable()
    table.title=f"Bottom values {column1} vs {column2}"
    table.field_names = cols

# Find the row with the maximum GDP value for each continent
#max_gdp_each_cont = df.groupby("Country")["Life expectancy"].idxmax()

# Extract and add the rows with the maximum GDP values to the table
    for index, row in cols.iterrows():
        table.add_row([row[column1], row[column2]])

    # Set the float format for the specified column
    table.float_format[column2] = ".2"


    return table

# **Data Visualization**

In [30]:
#Summary of Statistical Information
df.describe().T.sort_index()

,count,mean,std,min,25%,50%,75%,max
Agricultural Land( %),188.00,39.12,21.78,0.60,21.70,39.60,55.38,82.60
Agricultural Land(Km2),188.00,263772.91,681294.68,6.44,6739.85,39552.43,223619.28,5393491.52
Armed Forces Per Capita,171.00,0.01,0.01,0.00,0.00,0.00,0.01,0.06
Armed Forces size,171.00,159274.85,380628.80,0.00,11000.00,31000.00,142000.00,3031000.00
Birth Rate,189.00,20.21,9.95,5.90,11.30,17.95,28.75,46.08
CPI,178.00,190.46,397.95,99.03,113.89,125.34,157.27,4583.71
CPI Change (%),179.00,6.72,24.45,-4.30,1.00,2.30,4.25,254.90
Calling Code,194.00,360.55,323.24,1.00,82.50,255.50,506.75,1876.00
Co2-Emissions,188.00,177799.24,838790.27,11.00,2304.25,12303.00,63884.25,9893038.00
Co2-Emissions Per Capita,188.00,0.00,0.01,0.00,0.00,0.00,0.01,0.04


In [31]:
fig = px.sunburst(
    df,
    path=["Continent", "Country"],
    values="Population",
    color="Population_Percentage",
    color_continuous_scale="RdBu",
    color_continuous_midpoint=2.1,
    width=800,
    height=700
)

fig.update_traces(hovertemplate="<b>%{label}</b><br>Population: %{value}<br>Population_Percentage:%{customdata:.2f}",)
 
fig.update_layout(
    title="Population",
    title_font={"size": 20},
    margin=dict(t=50, b=50, l=0, r=0),
)
fig.show()


In [32]:
df_sorted = df.sort_values('Population', ascending=False)
top_n = 2

top_countries = df_sorted.head(top_n)

other_population = df_sorted.iloc[top_n:]['Population'].sum()

new_df = pd.concat([top_countries, pd.DataFrame({'Country': ['Others'], 'Population': [other_population]})])

colors = ['#FF0000', '#FFFF00', '#00FF00']

fig = go.Figure(data=[go.Pie(labels=new_df['Country'],
                             values=new_df['Population'],
                             textinfo="label+percent",
                             pull=[0, 0, 0],
                             showlegend=False,
                             marker_colors=colors,
                            )])


fig.update_traces(
    hole=0.4
)
fig.update_layout(
    title='Population distribution of Countries',
    template='gridon',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    annotations=[
        dict(
            text="Almost Half of the Earth's <br>Population lives in <br> India & China alone!!", 
            x=0.5, y=0.5, 
            font_size=13,
            showarrow=False
        )
    ]
)

fig.show()

In [33]:
choroplethMap("Population", "RdBu", False)

In [34]:
Tops = Top_Values("Country","Population",10)
Bots = Bottom_Values("Country","Population",10)

# Get the rows as strings
tops_rows = Tops.get_string().splitlines()
bots_rows = Bots.get_string().splitlines()

# Combine the tables side by side
combined_table = ""
for top_row, bot_row in zip(tops_rows, bots_rows):
    combined_table += f"{top_row}\t{bot_row}\n"

# Print the combined table
print(combined_table)

+----------------------------------+	+-------------------------------------+
| Top values Country vs Population |	| Bottom values Country vs Population |
+----------------+-----------------+	+-----------------------+-------------+
|    Country     |    Population   |	|        Country        |  Population |
+----------------+-----------------+	+-----------------------+-------------+
|     China      |  1397715000.00  |	|      Vatican City     |    836.00   |
|     India      |  1366417754.00  |	|         Nauru         |   10084.00  |
| United States  |   328239523.00  |	|         Tuvalu        |   11646.00  |
|   Indonesia    |   270203917.00  |	|         Palau         |   18233.00  |
|    Pakistan    |   216565318.00  |	|       San Marino      |   33860.00  |
|     Brazil     |   212559417.00  |	|     Liechtenstein     |   38019.00  |
|    Nigeria     |   200963599.00  |	|         Monaco        |   38964.00  |
|   Bangladesh   |   167310838.00  |	| Saint Kitts and Nevis |   52823.00  |

In [35]:
choroplethMap("GDP", "RdBu", False)

In [36]:
Tops = Top_Values("Country","GDP",10)
Bots = Bottom_Values("Country","GDP",10)

# Get the rows as strings
tops_rows = Tops.get_string().splitlines()
bots_rows = Bots.get_string().splitlines()

# Combine the tables side by side
combined_table = ""
for top_row, bot_row in zip(tops_rows, bots_rows):
    combined_table += f"{top_row}\t{bot_row}\n"

# Print the combined table
print(combined_table)

+------------------------------------+	+-------------------------------------------------+
|     Top values Country vs GDP      |	|           Bottom values Country vs GDP          |
+----------------+-------------------+	+----------------------------------+--------------+
|    Country     |        GDP        |	|             Country              |     GDP      |
+----------------+-------------------+	+----------------------------------+--------------+
| United States  | 21427700000000.00 |	|              Tuvalu              | 47271463.00  |
|     China      | 19910000000000.00 |	|              Nauru               | 133000000.00 |
|     Japan      |  5081769542380.00 |	|             Kiribati             | 194647202.00 |
|    Germany     |  3845630030824.00 |	|         Marshall Islands         | 221278000.00 |
| United Kingdom |  2827113184696.00 |	|              Palau               | 283994900.00 |
|     France     |  2715518274227.00 |	|  Federated States of Micronesia  | 401932279.00 |

In [37]:
choroplethMap("GDP Per Capita", "RdBu", False)

In [38]:

Tops = Top_Values("Country","GDP Per Capita",10)
Bots = Bottom_Values("Country","GDP Per Capita",10)

# Get the rows as strings
tops_rows = Tops.get_string().splitlines()
bots_rows = Bots.get_string().splitlines()

# Combine the tables side by side
combined_table = ""
for top_row, bot_row in zip(tops_rows, bots_rows):
    combined_table += f"{top_row}\t{bot_row}\n"

# Print the combined table
print(combined_table)


+--------------------------------------+	+-------------------------------------------+
| Top values Country vs GDP Per Capita |	|  Bottom values Country vs GDP Per Capita  |
+---------------------+----------------+	+--------------------------+----------------+
|       Country       | GDP Per Capita |	|         Country          | GDP Per Capita |
+---------------------+----------------+	+--------------------------+----------------+
|        Monaco       |   184396.99    |	|         Burundi          |     261.25     |
|    Liechtenstein    |   172357.47    |	|         Somalia          |     305.69     |
|      Luxembourg     |   110172.37    |	|         Eritrea          |     326.06     |
|     Switzerland     |    81993.73    |	|          Malawi          |     411.55     |
| Republic of Ireland |    77629.99    |	|          Sudan           |     441.51     |
|        Norway       |    75419.63    |	| Central African Republic |     467.91     |
|       Iceland       |    66944.83    |	| 

In [39]:
choroplethMap("Armed Forces size", "RdBu", False)

In [40]:
Tops = Top_Values("Country","Armed Forces size",10)
Bots = Bottom_Values("Country","Armed Forces size",10)

# Get the rows as strings
tops_rows = Tops.get_string().splitlines()
bots_rows = Bots.get_string().splitlines()

# Combine the tables side by side
combined_table = ""
for top_row, bot_row in zip(tops_rows, bots_rows):
    combined_table += f"{top_row}\t{bot_row}\n"

# Print the combined table
print(combined_table)

+-----------------------------------------+	+--------------------------------------------+
| Top values Country vs Armed Forces size |	| Bottom values Country vs Armed Forces size |
+-----------------+-----------------------+	+-----------------------+--------------------+
|     Country     |   Armed Forces size   |	|        Country        | Armed Forces size  |
+-----------------+-----------------------+	+-----------------------+--------------------+
|      India      |       3031000.00      |	|       Seychelles      |        0.00        |
|      China      |       2695000.00      |	|  Antigua and Barbuda  |        0.00        |
|   North Korea   |       1469000.00      |	|         Haiti         |        0.00        |
|      Russia     |       1454000.00      |	|         Libya         |        0.00        |
|  United States  |       1359000.00      |	|        Iceland        |        0.00        |
|     Pakistan    |       936000.00       |	|       Cape Verde      |      1000.00       |

In [41]:
choroplethMap("Fertility Rate", "RdBu", False)

In [42]:
Tops = Top_Values("Country","Fertility Rate",10)
Bots = Bottom_Values("Country","Fertility Rate",10)

# Get the rows as strings
tops_rows = Tops.get_string().splitlines()
bots_rows = Bots.get_string().splitlines()

# Combine the tables side by side
combined_table = ""
for top_row, bot_row in zip(tops_rows, bots_rows):
    combined_table += f"{top_row}\t{bot_row}\n"

# Print the combined table
print(combined_table)

+---------------------------------------------------+	+-----------------------------------------+
|        Top values Country vs Fertility Rate       |	| Bottom values Country vs Fertility Rate |
+----------------------------------+----------------+	+------------------------+----------------+
|             Country              | Fertility Rate |	|        Country         | Fertility Rate |
+----------------------------------+----------------+	+------------------------+----------------+
|              Niger               |      6.91      |	|      South Korea       |      0.98      |
|             Somalia              |      6.07      |	|       Singapore        |      1.14      |
| Democratic Republic of the Congo |      5.92      |	|         Malta          |      1.23      |
|               Mali               |      5.88      |	|        Moldova         |      1.26      |
|               Chad               |      5.75      |	|       San Marino       |      1.26      |
|              Angol

In [43]:
choroplethMap("Co2-Emissions", "RdBu", False)

In [44]:
Tops = Top_Values("Country","Co2-Emissions",10)
Bots = Bottom_Values("Country","Co2-Emissions",10)

# Get the rows as strings
tops_rows = Tops.get_string().splitlines()
bots_rows = Bots.get_string().splitlines()

# Combine the tables side by side
combined_table = ""
for top_row, bot_row in zip(tops_rows, bots_rows):
    combined_table += f"{top_row}\t{bot_row}\n"

# Print the combined table
print(combined_table)

+-------------------------------------+	+------------------------------------------------+
| Top values Country vs Co2-Emissions |	|     Bottom values Country vs Co2-Emissions     |
+------------------+------------------+	+--------------------------------+---------------+
|     Country      |  Co2-Emissions   |	|            Country             | Co2-Emissions |
+------------------+------------------+	+--------------------------------+---------------+
|      China       |    9893038.00    |	|             Tuvalu             |     11.00     |
|  United States   |    5006302.00    |	|         Liechtenstein          |     51.00     |
|      India       |    2407672.00    |	|            Kiribati            |     66.00     |
|      Russia      |    1732027.00    |	|     Sao Tome and Principe      |     121.00    |
|      Japan       |    1135886.00    |	|             Tonga              |     128.00    |
|     Germany      |    727973.00     |	|        Marshall Islands        |     143.00    |

In [45]:
choroplethMap("Agricultural Land( %)", "RdBu", False)

In [46]:
Tops = Top_Values("Country","Agricultural Land( %)",10)
Bots = Bottom_Values("Country","Agricultural Land( %)",10)

# Get the rows as strings
tops_rows = Tops.get_string().splitlines()
bots_rows = Bots.get_string().splitlines()

# Combine the tables side by side
combined_table = ""
for top_row, bot_row in zip(tops_rows, bots_rows):
    combined_table += f"{top_row}\t{bot_row}\n"

# Print the combined table
print(combined_table)

+---------------------------------------------+	+------------------------------------------------+
| Top values Country vs Agricultural Land( %) |	| Bottom values Country vs Agricultural Land( %) |
+----------------+----------------------------+	+--------------------+---------------------------+
|    Country     |   Agricultural Land( %)    |	|      Country       |   Agricultural Land( %)   |
+----------------+----------------------------+	+--------------------+---------------------------+
|    Uruguay     |           82.60            |	|      Suriname      |            0.60           |
|  Saudi Arabia  |           80.80            |	|     Singapore      |            0.90           |
|   Kazakhstan   |           80.40            |	|    The Bahamas     |            1.40           |
|  South Africa  |           79.80            |	|  Papua New Guinea  |            2.60           |
|    Burundi     |           79.20            |	|       Brunei       |            2.70           |
|    Niger

In [47]:
choroplethMap("Gross tertiary education enrollment (%)", "RdBu", False)

In [48]:
Tops = Top_Values("Country","Gross tertiary education enrollment (%)",10)
Bots = Bottom_Values("Country","Gross tertiary education enrollment (%)",10)

# Get the rows as strings
tops_rows = Tops.get_string().splitlines()
bots_rows = Bots.get_string().splitlines()

# Combine the tables side by side
combined_table = ""
for top_row, bot_row in zip(tops_rows, bots_rows):
    combined_table += f"{top_row}\t{bot_row}\n"

# Print the combined table
print(combined_table)

+---------------------------------------------------------------+	+--------------------------------------------------------------------+
| Top values Country vs Gross tertiary education enrollment (%) |	|  Bottom values Country vs Gross tertiary education enrollment (%)  |
+----------------+----------------------------------------------+	+--------------------------+-----------------------------------------+
|    Country     |   Gross tertiary education enrollment (%)    |	|         Country          | Gross tertiary education enrollment (%) |
+----------------+----------------------------------------------+	+--------------------------+-----------------------------------------+
|     Greece     |                    136.60                    |	|          Malawi          |                   0.80                  |
|   Australia    |                    113.10                    |	|          Haiti           |                   1.10                  |
|    Grenada     |                    104

In [49]:
choroplethMap("Life expectancy", "RdBu", False)

In [50]:
Tops = Top_Values("Country","Life expectancy",10)
Bots = Bottom_Values("Country","Life expectancy",10)

# Get the rows as strings
tops_rows = Tops.get_string().splitlines()
bots_rows = Bots.get_string().splitlines()

# Combine the tables side by side
combined_table = ""
for top_row, bot_row in zip(tops_rows, bots_rows):
    combined_table += f"{top_row}\t{bot_row}\n"

# Print the combined table
print(combined_table)

+---------------------------------------+	+--------------------------------------------+
| Top values Country vs Life expectancy |	|  Bottom values Country vs Life expectancy  |
+-----------------+---------------------+	+--------------------------+-----------------+
|     Country     |   Life expectancy   |	|         Country          | Life expectancy |
+-----------------+---------------------+	+--------------------------+-----------------+
|    San Marino   |        85.40        |	| Central African Republic |      52.80      |
|      Japan      |        84.20        |	|         Lesotho          |      53.70      |
|   Switzerland   |        83.60        |	|           Chad           |      54.00      |
|      Spain      |        83.30        |	|       Sierra Leone       |      54.30      |
|    Singapore    |        83.10        |	|         Nigeria          |      54.30      |
|  Liechtenstein  |        83.00        |	|         Somalia          |      57.10      |
|      Italy      |  

In [51]:
fig = px.treemap(
    df,
    path=["Continent", "Country"],
    values="GDP",
    color="GDP Per Country Per Cont.",
    color_continuous_scale="RdBu_r",
    hover_name="Country",
    width=1000,
    height=600,
    branchvalues="total",
)

fig.update_traces(
    hovertemplate="<b>%{label}</b><br>GDP: %{value}<br>GDP Per Country Per Cont.:%{customdata:.2f}",
)


fig.update_layout(
    title="GDP",
    title_font={"size": 20},
    margin=dict(t=50, b=0, l=0, r=0),
)
fig.show()

In [52]:
# Define the columns you want to include in the PrettyTable
cols = [
    "Continent",
    "Continent GDP",
    "Country",
    "GDP",
    "GDP Per Country Per Cont.",
]

# Create an instance of PrettyTable and set the field names
table = PrettyTable()
table.field_names = cols

# Find the row with the maximum GDP value for each continent
max_gdp_each_cont = df.groupby("Continent")["GDP"].idxmax()

# Extract and add the rows with the maximum GDP values to the table
for index in max_gdp_each_cont:
    row = df.loc[index]
    table.add_row([row[column] for column in cols])

# Print the table
print(table)

+---------------+------------------+---------------+------------------+---------------------------+
|   Continent   |  Continent GDP   |    Country    |       GDP        | GDP Per Country Per Cont. |
+---------------+------------------+---------------+------------------+---------------------------+
|     Africa    | 2440716805489.0  |    Nigeria    |  448120428859.0  |    0.18360197621092655    |
|      Asia     | 37631637700246.0 |     China     | 19910000000000.0 |     0.5290760970487831    |
|     Europe    | 21593697202087.0 |    Germany    | 3845630030824.0  |    0.17809039345297134    |
| North America | 24957824386416.0 | United States | 21427700000000.0 |     0.8585564057283226    |
|    Oceania    | 1638695166573.0  |   Australia   | 1392680589329.0  |     0.849871664808477     |
| South America | 3855537709300.0  |     Brazil    | 1839758040766.0  |     0.477172882093279     |
+---------------+------------------+---------------+------------------+---------------------------+


In [53]:
Land_By_Country = df.sort_values(by='Land Area(Km2)',ascending=False).head(10)


fig=px.scatter(Land_By_Country,x="Land Area(Km2)", y="Agricultural Land(Km2)",
               size="Agricultural Land( %)",
               color="Country",)

# Enable interactive legend functionality
fig.update_layout(legend=dict(itemclick='toggleothers'))

# Set the chart title and axis labels
fig.update_layout(title_text='Land Area vs Agricultural Area')

fig.show()


In [54]:
fig=px.scatter(Land_By_Country,x="Land Area(Km2)", y="Forested Area(Km2)",
               size="Forested Area (%)",
               color="Country",)

# Enable interactive legend functionality
fig.update_layout(legend=dict(itemclick='toggleothers'))

# Set the chart title and axis labels
fig.update_layout(title_text='Land Area vs Forest Area')

fig.show()

In [55]:
fig=px.scatter(Land_By_Country,x="Agricultural Land(Km2)", y="Forested Area(Km2)",
               size="Land Area(Km2)",
               color="Country",)

# Enable interactive legend functionality
fig.update_layout(legend=dict(itemclick='toggleothers'))

# Set the chart title and axis labels
fig.update_layout(title_text='Agricultural Area vs Forest Area')


fig.show()

In [56]:
GDP_By_Country = df.sort_values(by='GDP',ascending=False).head(10)

fig=make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(
            name="GDP",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["GDP"],
            mode='lines',
            showlegend=True,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=1,col=1,
        )

fig.add_trace(go.Scatter(
            name="CO2-Emissions",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["Co2-Emissions"],
            mode='lines',
            showlegend=True,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=2,col=1,
        )
fig.add_trace(go.Scatter(
            name="GDP Per Capita",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["GDP Per Capita"],
            mode='lines',
            showlegend=True,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=3,col=1,
        )

# Hide x-axis labels of the first subplot
fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.update_xaxes(showticklabels=False, row=2, col=1)

fig.show()

In [57]:
GDP_By_Country = df.sort_values(by='GDP',ascending=False).head(10)

fig=make_subplots(rows=6, cols=1)
fig.add_trace(go.Scatter(
            name="Birth Rate",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["Birth Rate"],
            mode='lines',
            showlegend=True,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=1,col=1,
        )

fig.add_trace(go.Scatter(
            name="Fertility Rate",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["Fertility Rate"],
            mode='lines',
            showlegend=True,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=2,col=1,
        )
fig.add_trace(go.Scatter(
            name="Infant mortality",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["Infant mortality"],
            mode='lines',
            showlegend=True,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=3,col=1,
        )
fig.add_trace(go.Scatter(
            name="Maternal mortality ratio",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["Maternal mortality ratio"],
            mode='lines',
            showlegend=True,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=4,col=1,
        )
fig.add_trace(go.Scatter(
            name="Life expectancy",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["Life expectancy"],
            mode='lines',
            showlegend=True,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=5,col=1,
        )
fig.add_trace(go.Scatter(
            name="GDP Per Capita",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["GDP Per Capita"],
            mode='lines',
            showlegend=True,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=6,col=1,
        )

# Hide x-axis labels of the first subplot
fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.update_xaxes(showticklabels=False, row=2, col=1)
fig.update_xaxes(showticklabels=False, row=3, col=1)
fig.update_xaxes(showticklabels=False, row=4, col=1)
fig.update_xaxes(showticklabels=False, row=5, col=1)

fig.show()

In [58]:
GDP_By_Country = df.sort_values(by='GDP',ascending=False).head(10)

fig=make_subplots(rows=3, cols=1, subplot_titles=("Gross Primary Educ. Enroll (%)", "Gross Tertiary Educ. Enroll (%)", "GDP Per Capita"))
fig.add_trace(go.Scatter(
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["Gross primary education enrollment (%)"],
            mode='lines',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=1,col=1,
        )

fig.add_trace(go.Scatter(
            #name="Gross Tertiary Educ. Enroll (%)",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["Gross tertiary education enrollment (%)"],
            mode='lines',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=2,col=1,
        )
fig.add_trace(go.Scatter(
            #name="GDP Per Capita",
            x=GDP_By_Country["Country"], 
            y=GDP_By_Country["GDP Per Capita"],
            mode='lines',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=3,col=1,
        )

# Hide x-axis labels of the first subplot
fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.update_xaxes(showticklabels=False, row=2, col=1)

fig.show()

In [59]:
df_by_Continent = df[['Continent','Continent GDP']].groupby(['Continent'], as_index=False).sum()
fig = px.bar(df_by_Continent, x='Continent', y='Continent GDP', title='GDP by Continent', 
             hover_data=['Continent', 'Continent GDP'], 
             labels={'Continent':'Continent', 'Continent GDP':'GDP'},
            color='Continent')

fig.show()

In [60]:
df_by_Continent = df[['Continent','Cont_Density']].groupby(['Continent'], as_index=False).sum()
fig = px.bar(df_by_Continent, x='Continent', y='Cont_Density', title='Density per Continent', 
             hover_data=['Continent', 'Cont_Density'], 
             labels={'Continent':'Continent', 'Cont_Density':'Density'},
            color='Continent')

fig.show()

In [61]:
df_by_Continent = df[['Continent','Cont_ArmedForcesSize']].groupby(['Continent'], as_index=False).sum()
fig = px.bar(df_by_Continent, x='Continent', y='Cont_ArmedForcesSize', title='FFAA per Continent', 
            hover_data=['Continent', 'Cont_ArmedForcesSize'], 
            labels={'Continent':'Continent', 'Cont_ArmedForcesSize':'FFAA'},
            color='Continent')

fig.show()

In [62]:

df_by_Continent = df[['Continent','Cont_Co2-Emissions']].groupby(['Continent'], as_index=False).sum()
fig = px.bar(df_by_Continent, x='Continent', y='Cont_Co2-Emissions', title='Co2 per Continent', 
            hover_data=['Continent', 'Cont_Co2-Emissions'], 
            labels={'Continent':'Continent', 'Cont_Co2-Emissions':'Co2'},
            color='Continent')

fig.show()

In [63]:
df_by_Continent = df[['Continent', 'Cont_Population', 'Cont_Density', 'Continent GDP']].groupby(['Continent'], as_index=False).sum()

# Create subplots
fig = make_subplots(rows=3, cols=1, subplot_titles=("Population", "Density", "GDP"))

# Add traces for each subplot
fig.add_trace(
    go.Scatter(
        #name="Cont_Population",
        x=df_by_Continent["Continent"],
        y=df_by_Continent["Cont_Population"],
        mode='lines',
        showlegend=False,
        marker=dict(
            opacity=0.5,
            showscale=False,
        )
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        #name="Cont_Density",
        x=df_by_Continent["Continent"],
        y=df_by_Continent["Cont_Density"],
        mode='lines',
        showlegend=False,
        marker=dict(
            opacity=0.5,
            showscale=False,
        )
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        #name="Continent GDP",
        x=df_by_Continent["Continent"],
        y=df_by_Continent["Continent GDP"],
        mode='lines',
        showlegend=False,
        marker=dict(
            opacity=0.5,
            showscale=False,
        )  ),
    row=3, col=1
)

# Update x-axis labels and title for the bottom subplot
#fig.update_xaxes(title_text="Continent", row=3, col=1)

fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.update_xaxes(showticklabels=False, row=2, col=1)


# Update y-axis titles
#fig.update_yaxes(title_text="Population", row=1, col=1)
#fig.update_yaxes(title_text="Density", row=2, col=1)
#fig.update_yaxes(title_text="GDP", row=3, col=1)

# Update layout
fig.update_layout(title_text="Continent Statistics")
fig.show()

In [64]:
df_by_Continent = df[['Continent','Urban Population Ratio','Forested Area(Km2)','Agricultural Land(Km2)','Agricultural Land( %)','Land Area(Km2)', 'Continent GDP']].groupby(['Continent'], as_index=False).sum()

# Create subplots
fig = make_subplots(rows=6, cols=1, subplot_titles=("Urban Pop. Ratio","Forested Area(Km2)","Agricultural Land(Km2)","Agricultural Land( %)","Land Area(Km2)", "GDP"))

# Add traces for each subplot
fig.add_trace(
    go.Scatter(
        #name="Cont_Population",
        x=df_by_Continent["Continent"],
        y=df_by_Continent["Urban Population Ratio"],
        mode='lines',
        showlegend=False,
        marker=dict(
            opacity=0.5,
            showscale=False,
        )
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        #name="Cont_Density",
        x=df_by_Continent["Continent"],
        y=df_by_Continent["Forested Area(Km2)"],
        mode='lines',
        showlegend=False,
        marker=dict(
            opacity=0.5,
            showscale=False,
        )
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        #name="Continent GDP",
        x=df_by_Continent["Continent"],
        y=df_by_Continent["Agricultural Land(Km2)"],
        mode='lines',
        showlegend=False,
        marker=dict(
            opacity=0.5,
            showscale=False,
        )  ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        #name="Continent GDP",
        x=df_by_Continent["Continent"],
        y=df_by_Continent["Agricultural Land( %)"],
        mode='lines',
        showlegend=False,
        marker=dict(
            opacity=0.5,
            showscale=False,
        )  ),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(
        #name="Continent GDP",
        x=df_by_Continent["Continent"],
        y=df_by_Continent["Land Area(Km2)"],
        mode='lines',
        showlegend=False,
        marker=dict(
            opacity=0.5,
            showscale=False,
        )  ),
    row=5, col=1
)

fig.add_trace(
    go.Scatter(
        #name="Continent GDP",
        x=df_by_Continent["Continent"],
        y=df_by_Continent["Continent GDP"],
        mode='lines',
        showlegend=False,
        marker=dict(
            opacity=0.5,
            showscale=False,
        )  ),
    row=6, col=1
)

# Update x-axis labels and title for the bottom subplot
#fig.update_xaxes(title_text="Continent", row=3, col=1)

fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.update_xaxes(showticklabels=False, row=2, col=1)
fig.update_xaxes(showticklabels=False, row=3, col=1)
fig.update_xaxes(showticklabels=False, row=4, col=1)
fig.update_xaxes(showticklabels=False, row=5, col=1)



# Update y-axis titles
#fig.update_yaxes(title_text="Population", row=1, col=1)
#fig.update_yaxes(title_text="Density", row=2, col=1)
#fig.update_yaxes(title_text="GDP", row=3, col=1)

# Update layout
fig.update_layout(title_text="Continent Statistics")
fig.show()

In [65]:
df_by_Continent = df[['Continent','Gross primary education enrollment (%)','Gross tertiary education enrollment (%)', 'Continent GDP']].groupby(['Continent'], as_index=False).mean()

# Create subplots
#fig = make_subplots(rows=6, cols=1, subplot_titles=("Urban Pop. Ratio","Forested Area(Km2)","Agricultural Land(Km2)","Agricultural Land( %)","Land Area(Km2)", "GDP"))

fig=make_subplots(rows=3, cols=1, subplot_titles=("Gross Primary Educ. Enroll (%)", "Gross Tertiary Educ. Enroll (%)", "Continent GDP"))
fig.add_trace(go.Scatter(
            x=df_by_Continent["Continent"], 
            y=df_by_Continent["Gross primary education enrollment (%)"],
            mode='lines',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=1,col=1,
        )

fig.add_trace(go.Scatter(
            #name="Gross Tertiary Educ. Enroll (%)",
            x=df_by_Continent["Continent"], 
            y=df_by_Continent["Gross tertiary education enrollment (%)"],
            mode='lines',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=2,col=1,
        )
fig.add_trace(go.Scatter(
            #name="GDP Per Capita",
            x=df_by_Continent["Continent"], 
            y=df_by_Continent["Continent GDP"],
            mode='lines',
            showlegend=False,
            marker=dict(
                opacity=0.5,
                showscale=False,
            )),row=3,col=1,
        )

# Hide x-axis labels of the first subplot
fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.update_xaxes(showticklabels=False, row=2, col=1)

fig.show()